In [2]:
import requests
import csv
import time
import pandas as pd
from osm_script import osm_extractor_groups, fetch_osm_region

In [3]:
#Several APIs available 

main_overpass_api = "https://overpass-api.de/api/interpreter"
lz4_overpass_api='https://lz4.overpass-api.de/api/interpreter'
osm_overpass_api='https://overpass.openstreetmap.ru/api/interpreter'

BUNDES_GROUPS = [
    ["Baden-Württemberg"],
    ["Bayern"],
    ["Berlin", "Bremen"],
    ["Brandenburg"],
    ["Hamburg"],
    ["Hessen","Mecklenburg-Vorpommern","Rheinland-Pfalz"],
    ["Niedersachsen"],
    ["Nordrhein-Westfalen"],
    ["Saarland","Sachsen","Sachsen-Anhalt", "Schleswig-Holstein","Thüringen"]
]

In [4]:
df_raw=osm_extractor_groups(BUNDES_GROUPS)

In [6]:
len(df_raw)

4604

In [16]:
df_raw = df_raw.replace(r'^\s*$', pd.NA, regex=True)

In [17]:
df_raw.sample(10) # raw results

,osm_id,region,type,name,street,housenumber,postcode,city,lat,lon,phone,email,website,amenity
4353,8440445795,Thüringen,node,"Bad Liebenstein, AWO Pflegeheim",<NA>,<NA>,<NA>,<NA>,50.812898,10.343869,<NA>,<NA>,<NA>,<NA>
2355,477802228,Nordrhein-Westfalen,node,<NA>,<NA>,<NA>,<NA>,<NA>,50.543300,6.902158,<NA>,<NA>,<NA>,recycling
1819,1323593168,Rheinland-Pfalz,way,AWO,<NA>,<NA>,<NA>,<NA>,49.508546,7.352684,<NA>,<NA>,<NA>,parking
2874,90578336,Nordrhein-Westfalen,way,KaWo2 Haus 3,Kastanienweg,31,52074,Aachen,50.784413,6.052369,<NA>,<NA>,<NA>,<NA>
2897,110019764,Nordrhein-Westfalen,way,Arbeiterwohlfahrt - Altenwohnungen,Dortmunder Straße,144a,45731,Waltrop,51.623091,7.415020,<NA>,<NA>,<NA>,<NA>
4300,7604310,Schleswig-Holstein,relation,Wandern mit Herz,<NA>,<NA>,<NA>,<NA>,54.627840,9.774961,<NA>,<NA>,<NA>,<NA>
3766,11559820690,Sachsen,node,AWO Sozialstation,Leipziger Straße,51,<NA>,<NA>,51.358563,12.295001,+49 341 4511434,<NA>,<NA>,community_centre
3489,1225214236,Nordrhein-Westfalen,way,Kita Drachennest,<NA>,<NA>,<NA>,<NA>,51.129508,6.448759,<NA>,<NA>,https://awo.mg/gut-heranwachsen/awo-kitas/,kindergarten
2692,11103478595,Nordrhein-Westfalen,node,Kindertagesstätte Deutzer Wiese,Gummersbacher Straße,31a,50679,Köln,50.938850,6.987140,<NA>,deutzer-wiese@awo-koeln.de,https://awo-koeln.de/kinder-jugend-familie/kin...,kindergarten
2816,52708982,Nordrhein-Westfalen,way,AWO,Kleybredde,32,44149,Dortmund,51.487694,7.373917,<NA>,<NA>,<NA>,<NA>


In [18]:
df_raw.region.value_counts()

region
Nordrhein-Westfalen       1236
Bayern                     679
Niedersachsen              513
Thüringen                  300
Baden-Württemberg          299
Sachsen                    281
Hessen                     237
Schleswig-Holstein         190
Brandenburg                163
Mecklenburg-Vorpommern     156
Sachsen-Anhalt             142
Saarland                   121
Rheinland-Pfalz             94
Berlin                      84
Bremen                      69
Hamburg                     40
Name: count, dtype: int64

In [19]:
df_raw.isnull().sum()

osm_id            0
region            0
type              0
name            413
street         1705
housenumber    1740
postcode       1825
city           1842
lat               0
lon               0
phone          3138
email          3827
website        2846
amenity        1308
dtype: int64

Next: 
    - removing findings which are not AWO facilities 
    - comparing with current Einrichtungsdatenbank 
    - comaring with domains .xlsx 
    - ...

In [ ]:
pattern_upper = r"\bAWO\b" # remove false positives WIP
has_word = df['name'].str.contains(pattern_word, case=False, na=False)
#has_any = big['name'].str.contains("awo", case=False, na=False)
mask = has_word|(~has_any)
df_filtered = df.loc[has_word].reset_index(drop=True)

In [22]:
name_datetime = time.strftime("%Y%m%d-%H%M%S")
df_raw.to_csv(f"awo_{name_datetime}_osm_raw.csv", index=False, encoding='utf-8')

In [20]:
jawoll = df_raw[df_raw['name']].str.contains("jawoll")
jawoll.shape

KeyError: 'None of [Index([                                         \'AWO-Kindertagesstätte Wiesenzwerg\',\n                                                                              \'AWO\',\n                                                                   \'Bawo\'s Kitchen\',\n                                                                      \'AWO-Gebäude\',\n       \'AWO Emma-Fackler-Schulkindergarten für körper- und sprachbehinderte Kinder\',\n                          \'AWO Kindertagesstätte "les explorateurs -die Entdecker"\',\n                                                          \'AWO Walderholungsstätte\',\n                                                              \'Kawohs Fischerhütte\',\n                                                     \'AWO Kita Am Weinbrennerplatz\',\n                                                                           \'ascawo\',\n       ...\n                                                             \'Marker Wiesenwichtel\',\

In [ ]:
#domains1=domains.assign(InOsm=domains.domain.isin(df.website).astype(int))